<a href="https://colab.research.google.com/github/ZainAfzalHashmi/Hacthon/blob/main/Encrypt_zain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install streamlit cryptography # cryptography for encryption
!npm install localtunnel # for exposing Streamlit app in Colab

⠙⠹⠸⠼⠴⠦
up to date, audited 23 packages in 684ms
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠦

In [8]:
%%writefile app.py
import streamlit as st
from cryptography.fernet import Fernet, InvalidToken
import base64
import os
import time

# --- Configuration ---
# Hardcoded admin credentials for initial login (for demonstration ONLY)
ADMIN_USERNAME = "admin"
ADMIN_PASSWORD = "securepassword123"

MAX_LOGIN_ATTEMPTS = 3
SESSION_TIMEOUT_SECONDS = 300 # 5 minutes

# --- In-Memory Data Storage ---
# data_store format: {encrypted_data_id: {'encrypted_payload': encrypted_bytes}}
# Note: The actual encryption key is derived from the user's passkey and is NOT stored here.
#       This means you need the correct passkey to decrypt!
if 'data_store' not in st.session_state:
    st.session_state.data_store = {}

# --- Session Management for Login ---
if 'logged_in' not in st.session_state:
    st.session_state.logged_in = False
if 'login_attempts' not in st.session_state:
    st.session_state.login_attempts = 0
if 'last_activity_time' not in st.session_state:
    st.session_state.last_activity_time = time.time()

# --- Helper Functions for Encryption/Decryption ---

# Generate a key from a passkey using a simple KDF (for demonstration)
# In a real app, use a proper KDF like PBKDF2 with a salt and many iterations.
def derive_key_from_passkey(passkey: str):
    # This is a *very simple* key derivation. For production, use KDF like PBKDF2HMAC
    # with a salt and many iterations.
    key_bytes = passkey.encode('utf-8')
    # Pad or hash the key to ensure it's the correct length for Fernet (32 bytes base64)
    # A simple way for demo: take first 32 bytes of SHA256 hash.
    import hashlib
    hashed_passkey = hashlib.sha256(key_bytes).digest()
    return base64.urlsafe_b64encode(hashed_passkey)

def encrypt_data(data: str, passkey: str) -> bytes:
    key = derive_key_from_passkey(passkey)
    f = Fernet(key)
    encrypted_data = f.encrypt(data.encode('utf-8'))
    return encrypted_data

def decrypt_data(encrypted_data: bytes, passkey: str) -> str:
    key = derive_key_from_passkey(passkey)
    f = Fernet(key)
    decrypted_data = f.decrypt(encrypted_data).decode('utf-8')
    return decrypted_data

# --- Authentication Logic ---

def check_session_timeout():
    if time.time() - st.session_state.last_activity_time > SESSION_TIMEOUT_SECONDS:
        st.session_state.logged_in = False
        st.session_state.login_attempts = 0 # Reset attempts on timeout
        st.warning("Session timed out. Please log in again.")
        st.experimental_rerun() # Force a rerun to show login page

def login_page():
    st.title("🔒 Secure Data System Login")
    username = st.text_input("Username")
    password = st.text_input("Password", type="password")

    if st.button("Login"):
        if username == ADMIN_USERNAME and password == ADMIN_PASSWORD:
            st.session_state.logged_in = True
            st.session_state.login_attempts = 0
            st.session_state.last_activity_time = time.time()
            st.success("Login successful!")
            st.experimental_rerun() # Rerun to show main app
        else:
            st.session_state.login_attempts += 1
            st.error("Invalid credentials.")
            if st.session_state.login_attempts >= MAX_LOGIN_ATTEMPTS:
                st.error("Too many failed attempts. Please try again later.")
                # In a real system, you'd implement a lockout time here.
                # For this demo, just force a re-render.
                st.session_state.logged_in = False
                st.session_state.login_attempts = 0
                st.stop() # Stop execution until manual rerun or new session
            else:
                st.info(f"Attempts remaining: {MAX_LOGIN_ATTEMPTS - st.session_state.login_attempts}")

def logout():
    st.session_state.logged_in = False
    st.session_state.login_attempts = 0
    st.session_state.data_store = {} # Clear in-memory data on logout for security
    st.success("Logged out successfully.")
    st.experimental_rerun()

# --- Main Application Logic ---
def main_app():
    st.session_state.last_activity_time = time.time() # Update activity time

    st.title("Secure In-Memory Data Storage")
    st.sidebar.header("Navigation")
    app_mode = st.sidebar.radio("Choose action", ["Store Data", "Retrieve Data", "View All IDs"])

    if st.sidebar.button("Logout"):
        logout()

    if app_mode == "Store Data":
        st.subheader("Store New Data")
        unique_id = st.text_input("Enter a unique ID for your data (e.g., 'MyBankDetails'):")
        data_to_store = st.text_area("Enter the data you want to store:")
        passkey_store = st.text_input("Enter a passkey to encrypt this data:", type="password", key="passkey_store")

        if st.button("Store Data"):
            if not unique_id or not data_to_store or not passkey_store:
                st.error("All fields are required!")
            else:
                try:
                    encrypted_data = encrypt_data(data_to_store, passkey_store)
                    st.session_state.data_store[unique_id] = {'encrypted_payload': encrypted_data}
                    st.success(f"Data stored successfully under ID: '{unique_id}'")
                    st.info("Remember your passkey! It's crucial for decryption.")
                    st.json({"Unique ID": unique_id, "Encrypted Payload (truncated)": encrypted_data[:50].decode() + "..."})
                except Exception as e:
                    st.error(f"Error storing data: {e}")

    elif app_mode == "Retrieve Data":
        st.subheader("Retrieve and Decrypt Data")
        unique_id_retrieve = st.selectbox(
            "Select an ID to retrieve:",
            [""] + list(st.session_state.data_store.keys())
        )
        passkey_retrieve = st.text_input("Enter the passkey to decrypt this data:", type="password", key="passkey_retrieve")

        if st.button("Decrypt Data"):
            if not unique_id_retrieve:
                st.error("Please select a data ID.")
            elif unique_id_retrieve not in st.session_state.data_store:
                st.error("Data ID not found.")
            elif not passkey_retrieve:
                st.error("Please enter the passkey.")
            else:
                encrypted_payload = st.session_state.data_store[unique_id_retrieve]['encrypted_payload']
                try:
                    decrypted_data = decrypt_data(encrypted_payload, passkey_retrieve)
                    st.success(f"Successfully decrypted data for ID: '{unique_id_retrieve}'")
                    st.code(decrypted_data, language='text')
                except InvalidToken:
                    st.error("Incorrect passkey! Decryption failed.")
                except Exception as e:
                    st.error(f"An unexpected error occurred during decryption: {e}")

    elif app_mode == "View All IDs":
        st.subheader("Stored Data IDs")
        if st.session_state.data_store:
            for i, uid in enumerate(st.session_state.data_store.keys()):
                st.write(f"{i+1}. **{uid}**")
            st.info("Note: Only IDs are shown. Data remains encrypted until retrieved with the correct passkey.")
        else:
            st.info("No data currently stored.")

# --- Application Flow Control ---
if not st.session_state.logged_in:
    login_page()
else:
    check_session_timeout()
    main_app()

Writing app.py
